### Goal: Prototype some QC plots for use with MultiQC

In [ ]:
%%bash

# Collect per-exon read counts from runs of the Dragen pipeline:
mkdir -p data

# From a recent run of the Twist Exome v2 panel on 8 Clinical blood samples:
(echo -ne "Chrom\tStart\tEnd\tGene\t" && ls /mnt/pns/vcfs/220426_A01718_0009_AHWVGHDRXY/Exome/Exome*A/*.target_bed_read_cov_report.bed | cut -f6 -d/ | sed -E 's/-|Twist//g; s/^D3|^SS/LCL_/' | paste -s) > data/twist_v2_exon_covg.txt
ls /mnt/pns/vcfs/220426_A01718_0009_AHWVGHDRXY/Exome/Exome*A/*.target_bed_read_cov_report.bed | xargs paste | cut -f1-4,$(seq -s, 6 8 62) | tail -n+2 >> data/twist_v2_exon_covg.txt

# Make a subset for the genes in the 220-gene heme panel:
cut -f1 ~/src/heme-panel-design/data/exon_targets_gene_list.txt > data/ucla_heme_220.txt
head -n1 data/twist_v2_exon_covg.txt > data/twist_v2_exon_covg_heme_genes.txt
grep -wf data/ucla_heme_220.txt data/twist_v2_exon_covg.txt >> data/twist_v2_exon_covg_heme_genes.txt

# From a recent run of our 218-gene GOAL-probe heme panel on 8 FFPE/Blood samples using KAPA LP+Cap kits:
(echo -ne "Chrom\tStart\tEnd\tGene\t" && ls /mnt/pns/vcfs/220426_A01718_0009_AHWVGHDRXY/Heme/*{E,F}/*.target_bed_read_cov_report.bed | cut -f7 -d/ | perl -pe 's/.*(\d)E$/$1_GOAL_50ng/; s/.*(\d)F$/$1_GOAL_100ng/;' | paste -s) > data/goal_kapa_heme_exon_covg.txt
ls /mnt/pns/vcfs/220426_A01718_0009_AHWVGHDRXY/Heme/*{E,F}/*.target_bed_read_cov_report.bed | xargs paste | cut -f1-4,$(seq -s, 6 8 62) | tail -n+2 >> data/goal_kapa_heme_exon_covg.txt

# From a recent run of our 218-gene GOAL-probe heme panel on 8 FFPE/Blood samples using Twist LP+Cap kits:
(echo -ne "Chrom\tStart\tEnd\tGene\t" && ls /mnt/pns/vcfs/220426_A01718_0009_AHWVGHDRXY/Heme/*{A,C}/*.target_bed_read_cov_report.bed | cut -f7 -d/ | perl -pe 's/.*(\d)A$/$1_GOAL_50ng/; s/.*(\d)C$/$1_GOAL_25ng/;' | paste -s) > data/goal_heme_exon_covg.txt
ls /mnt/pns/vcfs/220426_A01718_0009_AHWVGHDRXY/Heme/*{A,C}/*.target_bed_read_cov_report.bed | xargs paste | cut -f1-4,$(seq -s, 6 8 62) | tail -n+2 >> data/goal_heme_exon_covg.txt

# From a recent run of our 220-gene Twist-probe heme panel on 8 FFPE/Blood samples using Twist LP+Cap kits:
(echo -ne "Chrom\tStart\tEnd\tGene\t" && ls /mnt/pns/vcfs/220915_A01718_0013_AHWVHFDRXY/Heme/*{A,C}/*.target_bed_read_cov_report.bed | cut -f7 -d/ | perl -pe 's/.*(\d)A$/$1_Twist_50ng/; s/.*(\d)C$/$1_Twist_25ng/;' | paste -s) > data/ucla_heme_exon_covg.txt
ls /mnt/pns/vcfs/220915_A01718_0013_AHWVHFDRXY/Heme/*{A,C}/*.target_bed_read_cov_report.bed | xargs paste | cut -f1-4,$(seq -s, 6 8 62) | tail -n+2 >> data/ucla_heme_exon_covg.txt

In [ ]:
# Load necessary packages and data
import pandas as pd
import plotly.express as px
dfk = pd.read_csv("data/goal_kapa_heme_exon_covg.txt", sep='\t')
dfg = pd.read_csv("data/goal_heme_exon_covg.txt", sep='\t')
dfh = pd.read_csv("data/ucla_heme_exon_covg.txt", sep='\t')

# Convert read counts to read depth per exon, and then divide that by overall mean to get fold difference
dfk.iloc[:, list(range(4,12))] = dfk.iloc[:, list(range(4,12))].apply(lambda x: 150*x/(dfk.End - dfk.Start))
dfk.iloc[:, list(range(4,12))] = dfk.iloc[:, list(range(4,12))].apply(lambda x: x/x.mean())
dfg.iloc[:, list(range(4,12))] = dfg.iloc[:, list(range(4,12))].apply(lambda x: 150*x/(dfg.End - dfg.Start))
dfg.iloc[:, list(range(4,12))] = dfg.iloc[:, list(range(4,12))].apply(lambda x: x/x.mean())
dfh.iloc[:, list(range(4,12))] = dfh.iloc[:, list(range(4,12))].apply(lambda x: 150*x/(dfh.End - dfh.Start))
dfh.iloc[:, list(range(4,12))] = dfh.iloc[:, list(range(4,12))].apply(lambda x: x/x.mean())

In [ ]:
# Make a violin plot:
fig = px.violin(dfk, x=dfk.columns[list(range(4,12))], range_x=[-0.5,17], title="Exon depth uniformity of 218 heme genes using KAPA LP/Cap kits and GOAL probes", labels={"variable": "", "value": "Fold difference from mean of exon depths"}, width=1200, height=1200, template="seaborn", box=True, points=False)
fig.add_vrect(x0=0.5, x1=2, opacity=0.3)
fig.add_annotation(x=0.5, y=7.4, text="We want most exons within<br>0.5x-2x the mean exon depth", showarrow=True, arrowhead=4)
fig.update_layout(violingap=0, xaxis=dict(tickmode='linear', dtick=1), yaxis=dict(categoryorder="category descending"))
fig.show(renderer="png")

In [ ]:
# Make a violin plot:
fig = px.violin(dfg, x=dfg.columns[list(range(4,12))], range_x=[-0.5,17], title="Exon depth uniformity of 218 heme genes using Twist LP/Cap kits and GOAL probes", labels={"variable": "", "value": "Fold difference from mean of exon depths"}, width=1200, height=1200, template="seaborn", box=True, points=False)
fig.add_vrect(x0=0.5, x1=2, opacity=0.3)
fig.add_annotation(x=0.5, y=7.4, text="We want most exons within<br>0.5x-2x the mean exon depth", showarrow=True, arrowhead=4)
fig.update_layout(violingap=0, xaxis=dict(tickmode='linear', dtick=1), yaxis=dict(categoryorder="category descending"))
fig.show(renderer="png")

In [ ]:
# Make a violin plot:
fig = px.violin(dfh, x=dfh.columns[list(range(4,12))], range_x=[-0.5,17], title="Exon depth uniformity of 220 heme genes using Twist LP/Cap kits and Twist probes", labels={"variable": "", "value": "Fold difference from mean of exon depths"}, width=1200, height=1200, template="seaborn", box=True, points=False)
fig.add_vrect(x0=0.5, x1=2, opacity=0.3)
fig.add_annotation(x=0.5, y=7.4, text="We want most exons within<br>0.5x-2x the mean exon depth", showarrow=True, arrowhead=4)
fig.update_layout(violingap=0, xaxis=dict(tickmode='linear', dtick=1), yaxis=dict(categoryorder="category descending"))
fig.show(renderer="png")